### Data Processing

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
# Import Data
orders_df = pd.read_csv('olist_orders_dataset.csv')
order_items_df = pd.read_csv('olist_order_items_dataset.csv')
products_df = pd.read_csv('olist_products_dataset.csv')
customers_df = pd.read_csv('olist_customers_dataset.csv')
sellers_df = pd.read_csv('olist_sellers_dataset.csv')

### Part 1: Data Transformation

This part focuses on transforming the data into the appropriate structure, deriving relevant attributes, discarding irrelevant
data, and establishing relations through primary and foreign keys.

In [3]:
# Transform Data: -Restructure data into star schema format   -Create separate DataFrames for dimensions and facts

# Date dimension
date_dimension = pd.DataFrame({'date': pd.to_datetime(orders_df['order_purchase_timestamp']).dt.date.unique()})  # Ensure that date fields are parsed as datetime objects 
# Extract year, month, and day attributes from the order_purchase_timestamp
date_dimension['year'] = pd.to_datetime(date_dimension['date']).dt.year
date_dimension['month'] = pd.to_datetime(date_dimension['date']).dt.month
date_dimension['month_name'] = pd.to_datetime(date_dimension['date']).dt.strftime('%B')
date_dimension['day'] = pd.to_datetime(date_dimension['date']).dt.day
date_dimension['date_key'] = np.arange(len(date_dimension))


# 'xx_key' columns will serve as the PK for the dimension tables
product_dimension = pd.DataFrame({'product_key': products_df['product_id']})  # 'product_key' column will serve as the PK for the product dimension
seller_dimension = pd.DataFrame({'seller_key': sellers_df['seller_id']})
customer_dimension = pd.DataFrame({'customer_key': customers_df['customer_id']})
order_dimension = pd.DataFrame({'order_key': orders_df['order_id']})




In [4]:
# Merge with datasets dfs to include additional fields in the dimesnion tables:
product_dimension = pd.merge(product_dimension, products_df, how='left', left_on='product_key', right_on='product_id')
seller_dimension = pd.merge(seller_dimension, sellers_df, how='left', left_on='seller_key', right_on='seller_id')
customer_dimension = pd.merge(customer_dimension, customers_df, how='left', left_on='customer_key', right_on='customer_id')
order_dimension = pd.merge(order_dimension, orders_df, how='left', left_on='order_key', right_on='order_id')

# Generate a manual key for order_items: // since we don't have unique ids in order_items
order_items_df['order_item_unique_id'] = range(1, len(order_items_df) + 1)       
# Assuming that 'order_item_id' column is acttually the 'quantity' so rename the field
order_items_df.rename(columns={'order_item_id': 'quantity'}, inplace=True)
# Drop duplicates and keep the row with the maximum quantity:
order_items_df = order_items_df.sort_values(by=['order_id', 'product_id', 'quantity'], ascending=False) # first descendibg sort
order_items_df.drop_duplicates(subset=['order_id', 'product_id'], keep='first', inplace=True)    # and then keep the first occurence = max and delete the other duplicate rows

# Create the order_item dimension DataFrame
order_item_dimension = pd.DataFrame({'order_item_key': order_items_df['order_item_unique_id']})
order_item_dimension = pd.merge(order_item_dimension, order_items_df, how='left', left_on='order_item_key', right_on='order_item_unique_id')




# Keep only the needed attributes in the dimension tables to answer the analytical question, and discard the others.
order_dimension = order_dimension[['order_key', 'customer_id', 'order_purchase_timestamp']]
# Convert order date in order_dimension to year-month-day format
order_dimension['order_purchase_timestamp'] = pd.to_datetime(order_dimension['order_purchase_timestamp']).dt.date

order_item_dimension = order_item_dimension[['order_item_key', 'order_id', 'product_id', 'seller_id', 'price', 'quantity']]
# Rename the 'price' column to 'price_per_unit'
order_item_dimension.rename(columns={'price': 'price_per_unit'}, inplace=True)


product_dimension = product_dimension[['product_key', 'product_category_name']]
customer_dimension = customer_dimension[['customer_key']]
seller_dimension = seller_dimension[['seller_key', 'seller_state']]




### Part 2: Data Integration

This part involves connecting the datasets through shared attributes and establishing integration functions to connect entities if necessary.

In [5]:
# Connect datasets through shared attributes (e.g., customer_id) (by merging DataFrames: merging the fact table with dimesnions tables)

sales_fact = order_item_dimension


# Merging order items with orders
sales_fact = pd.merge(sales_fact, order_dimension, how='left', left_on='order_id', right_on='order_key') # We have to establish a relationship between the sales_fact df and the order_dimension df // here we are specifying the FKs



# Merging with customers
sales_fact = pd.merge(sales_fact, customer_dimension, how='left', left_on='customer_id', right_on='customer_key')



# Merging with products
sales_fact = pd.merge(sales_fact, product_dimension, how='left', left_on='product_id', right_on='product_key') 



# Merging with sellers
sales_fact = pd.merge(sales_fact, seller_dimension, how='left', left_on='seller_id', right_on='seller_key')

#column_names = sales_fact.columns
#print(column_names)


# Merge with date
sales_fact = pd.merge(sales_fact, date_dimension, how='left', left_on='order_purchase_timestamp', right_on='date')



# print(sales_fact.head())



### Part 3: Data Cleaning

This part assesses if the data is subject to quality issues and resolves them where possible.

In [6]:
# Assess data quality and resolve issues

# Check empty entries:
print("Number of entries with null order_key before removal:", sales_fact['order_key'].isnull().sum())
print("Number of entries with null product_key before removal:", sales_fact['product_key'].isnull().sum())
print("Number of entries with null product_category_name before removal:", sales_fact['product_category_name'].isnull().sum())
print("Number of entries with null customer_key before removal:", sales_fact['customer_key'].isnull().sum())
print("Number of entries with null seller_key before removal:", sales_fact['seller_key'].isnull().sum())
print("Number of entries with null seller_state before removal:", sales_fact['seller_state'].isnull().sum())
print("Number of entries with null price_per_unit before removal:", sales_fact['price_per_unit'].isnull().sum())
print("Number of entries with null quantity before removal:", sales_fact['quantity'].isnull().sum())

# we don't check date_key and order_item_unique_id since we previously generated them manually

# Remove rows where 'product_id' is null
sales_fact = sales_fact.dropna(subset=['order_key', 'order_item_key', 'product_key', 'product_category_name', 'customer_key', 'date_key', 'seller_key', 'seller_state', 'price_per_unit', 'quantity']) # NO NULL product_id has been found!


# Verify removal
print("Number of entries with null order_key after removal:", sales_fact['order_key'].isnull().sum())
print("Number of entries with null product_key after removal:", sales_fact['product_key'].isnull().sum())
print("Number of entries with null product_category_name after removal:", sales_fact['product_category_name'].isnull().sum())
print("Number of entries with null customer_key after removal:", sales_fact['customer_key'].isnull().sum())
print("Number of entries with null seller_key after removal:", sales_fact['seller_key'].isnull().sum())
print("Number of entries with null seller_state after removal:", sales_fact['seller_state'].isnull().sum())
print("Number of entries with null price_per_unit after removal:", sales_fact['price_per_unit'].isnull().sum())
print("Number of entries with null quantity after removal:", sales_fact['quantity'].isnull().sum())


# Remove null entries
order_item_dimension = order_item_dimension.dropna(subset=['order_item_key', 'order_id', 'product_id', 'seller_id', 'price_per_unit', 'quantity']) 
# make sure they're removed:
#print("Number of entries with null quantity after removal:", order_item_dimension['quantity'].isnull().sum())
#print("Number of entries with null price_per_unit after removal:", order_item_dimension['price_per_unit'].isnull().sum())
order_dimension = order_dimension.dropna(subset=['order_key', 'customer_id', 'order_purchase_timestamp'])
product_dimension = product_dimension.dropna(subset=['product_key', 'product_category_name']) 
date_dimension = date_dimension.dropna(subset=['date_key', 'date'])
customer_dimension = customer_dimension.dropna(subset=['customer_key']) 
seller_dimension = seller_dimension.dropna(subset=['seller_key', 'seller_state']) 






Number of entries with null order_key before removal: 0
Number of entries with null product_key before removal: 0
Number of entries with null product_category_name before removal: 1460
Number of entries with null customer_key before removal: 0
Number of entries with null seller_key before removal: 0
Number of entries with null seller_state before removal: 0
Number of entries with null price_per_unit before removal: 0
Number of entries with null quantity before removal: 0
Number of entries with null order_key after removal: 0
Number of entries with null product_key after removal: 0
Number of entries with null product_category_name after removal: 0
Number of entries with null customer_key after removal: 0
Number of entries with null seller_key after removal: 0
Number of entries with null seller_state after removal: 0
Number of entries with null price_per_unit after removal: 0
Number of entries with null quantity after removal: 0


In [7]:
# Check for duplicate rows based on key ids:
duplicates = sales_fact.duplicated(subset=['order_key', 'order_item_key', 'customer_key', 'product_key'], keep='first')  # remove duplicate rows where it's the same order same odrer_item by the same customer and same product simultaneously.
print(sales_fact[duplicates]) # must be empty (0 duplicates) since we already kept the rows with the max quantity in the part of Data Transformation


# Filter out the duplicates from the df:
sales_fact = sales_fact[~duplicates]


Empty DataFrame
Columns: [order_item_key, order_id, product_id, seller_id, price_per_unit, quantity, order_key, customer_id, order_purchase_timestamp, customer_key, product_key, product_category_name, seller_key, seller_state, date, year, month, month_name, day, date_key]
Index: []


### Data Export:

In [8]:
# Export the Transformed and cleaned data to CSV files = Dimensions tables + Fact table

# Keep only the keys in fact table:
sales_facts = sales_fact[['order_key', 'order_item_key', 'product_key', 'customer_key', 'date_key', 'seller_key']]
sales_facts.to_csv('Output/sales_fact.csv', index=False)

# Dimensions have the needed attributes
order_dimension.to_csv('Output/order_dimension.csv', index=False)
order_item_dimension.to_csv('Output/orderitem_dimension.csv', index=False)
product_dimension.to_csv('Output/product_dimension.csv', index=False)
customer_dimension.to_csv('Output/customer_dimension.csv', index=False)
seller_dimension.to_csv('Output/seller_dimension.csv', index=False)
date_dimension.to_csv('Output/date_dimension.csv', index=False)


In [9]:
# Make sure again we don't have any null value in all tables!
file_path = "Output/sales_fact.csv" 
df = pd.read_csv(file_path)

null_counts = df.isnull().sum()

print("Number of null values in each column in sales_fact:")
print(null_counts)


file_path = "Output/order_dimension.csv" 
df = pd.read_csv(file_path)

null_counts = df.isnull().sum()

print("Number of null values in each column in order_dimension:")
print(null_counts)

file_path = "Output/orderitem_dimension.csv" 
df = pd.read_csv(file_path)

null_counts = df.isnull().sum()

print("Number of null values in each column in orderitem_dimension:")
print(null_counts)

file_path = "Output/product_dimension.csv" 
df = pd.read_csv(file_path)

null_counts = df.isnull().sum()

print("Number of null values in each column in product_dimension:")
print(null_counts)

file_path = "Output/customer_dimension.csv" 
df = pd.read_csv(file_path)

null_counts = df.isnull().sum()

print("Number of null values in each column in customer_dimension:")
print(null_counts)

file_path = "Output/seller_dimension.csv" 
df = pd.read_csv(file_path)

null_counts = df.isnull().sum()

print("Number of null values in each column in seller_dimension:")
print(null_counts)

Number of null values in each column in sales_fact:
order_key         0
order_item_key    0
product_key       0
customer_key      0
date_key          0
seller_key        0
dtype: int64
Number of null values in each column in order_dimension:
order_key                   0
customer_id                 0
order_purchase_timestamp    0
dtype: int64
Number of null values in each column in orderitem_dimension:
order_item_key    0
order_id          0
product_id        0
seller_id         0
price_per_unit    0
quantity          0
dtype: int64
Number of null values in each column in product_dimension:
product_key              0
product_category_name    0
dtype: int64
Number of null values in each column in customer_dimension:
customer_key    0
dtype: int64
Number of null values in each column in seller_dimension:
seller_key      0
seller_state    0
dtype: int64
